In [1]:
import re
import numpy as np
import json
from nltk.tokenize import word_tokenize,sent_tokenize
from gensim.models import Word2Vec

In [10]:
def squaddata_processing():
    with open('/home/user/Autofill/data/dev-v1.1.json') as data_file:
        dataset = json.load(data_file)
    raw_data = []
    for data in dataset['data']:
        paragraphs = (data['paragraphs'])
        dct = {}
        for paragraph in paragraphs:
            dct['Passage'] = paragraph['context']
            qlist = []
            for questions in paragraph['qas']:
                qlist.append(questions['question'])
            dct['Question'] = list(set(qlist)) 
        raw_data.append(dct) 
    with open('/home/user/Autofill/data/squaddata.json','w') as outfile:
        json.dump(raw_data,outfile)    

In [11]:
def combined_file():
    with open('/home/user/Autofill/data/squaddata.json') as data_file:
        dataset1 = json.load(data_file)
    with open('/home/user/Autofill/data/wiki_text (1).json') as data_file:
        dataset2 = json.load(data_file)
    final_dict = {}    
    final_para = ""
    final_question = []
    for i in range(0,len(dataset1),1):
        para = (dataset1[i]['Passage'])
        final_para = final_para + " " + para
        for j in range(0,len(dataset1[i]['Question']),1):
            final_question.append(dataset1[i]['Question'][j])
    final_para = final_para + " "        
    for i in range(0,len(dataset2),1):
        para = (dataset2[i]['Passage'])
        final_para = final_para + " " +para
        for j in range(0,len(dataset2[i]['Question']),1):
            final_question.append(dataset2[i]['Question'][j])
    final_dict['Paragraph'] = final_para
    final_dict['Question'] = final_question
    final_data = []
    final_data.append(final_dict)
    with open('/home/user/Autofill/data/final_json_file.json','w') as outfile:
        json.dump(final_data,outfile)

In [18]:
def preprocessor(raw_text):
    # removes all the punctuations and retains only alphabets,digits,fullstop,apostrophe
    clean_text = re.sub(r'[^\w\'\.\+\-\=\*\^]',' ' ,raw_text)
    # sentence tokenize clean text
    sentences = sent_tokenize(clean_text)
    # replace full stops from each sentence in sentences list by null
    sentences = [sent.replace('.','') for sent in sentences]
    # word tokenize each sentence in sentences list
    tokenized_sentences = [word_tokenize(sent) for sent in sentences]
    # initialize word2vector model
    model = Word2Vec(sentences = tokenized_sentences, size = 5, window = 5, min_count = 1, workers = 4)
    # finding out the vocabulary of raw_text with index
    vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])
    # replacing each word of tokenized_sentences by coressponding vocabulary index
    indexed_sentences = [[vocab[word] for word in sent] for sent in tokenized_sentences]
    # storeing indexed_sentences in indexed.json file
    with open('/home/user/Autofill/data/indexed.json','w') as outfile:
        json.dump(indexed_sentences,outfile)
    # finding gensim weights
    weights = model.wv.syn0
    # storeing weights in wordembeddings.npz file
    np.save(open('/home/user/Autofill/data/wordembeddings.npz', 'wb'), weights)

In [12]:
squaddata_processing()

In [13]:
combined_file()

In [19]:
with open('/home/user/Autofill/data/final_json_file.json') as data_file:
        dataset = json.load(data_file)
passage_text = dataset[0]['Paragraph']
question_text = "" 
for quest in dataset[0]['Question']:
    question_text = question_text + " " + quest 
raw_text = passage_text + question_text
preprocessor(raw_text)        